In [17]:
# %% 
# ----------------------------
# Imports
# ----------------------------

import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import cv2
import tensorflow as tf
import os

# ----------------------------
# Optional: U2Net Loading (for segmentation)
# ----------------------------
U2NET_AVAILABLE = True  # Set True if you have U2Net and want segmentation

if U2NET_AVAILABLE:
    from u2net import U2NET  # Ensure U2Net.py is in the same folder
    u2net_model_path = "u2net.pth"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    u2net = U2NET(3,1)
    u2net.load_state_dict(torch.load(u2net_model_path, map_location=device))
    u2net.to(device)
    u2net.eval()

    def segment_person(image_path):
        image = Image.open(image_path).convert('RGB')
        transform = transforms.Compose([
            transforms.Resize((320,320)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        ])
        img_tensor = transform(image).unsqueeze(0).to(device)
        
        with torch.no_grad():
            d1,d2,d3,d4,d5,d6,d7 = u2net(img_tensor)
            mask = d1[:,0,:,:]
            mask = (mask - mask.min()) / (mask.max() - mask.min() + 1e-8)  # normalize 0-1
            mask = mask.squeeze().cpu().numpy()
            mask = cv2.resize(mask, (image.width, image.height))
            mask = (mask > 0.5).astype(np.uint8)

        img_np = np.array(image)
        img_np = img_np * mask[:,:,np.newaxis]
        segmented = Image.fromarray(img_np)
        return segmented

# ----------------------------
# Load the trained TensorFlow model
# ----------------------------
# model_path = "body_shape_model_simplex.h5"  
model_path ="final1.h5"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model not found at {model_path}")

model = tf.keras.models.load_model(model_path)
class_labels = ['Apple','Hourglass','InvertedTriangle','Rectangle']


In [22]:

# ----------------------------
# Classification function
# ----------------------------
def classify_body_shape(image_path, use_segmentation=False):
    """
    Args:
        image_path (str): Path to the image file
        use_segmentation (bool): If True, uses U2Net to segment person
    
    Returns:
        label (str): Predicted body shape
        confidence (float): Prediction confidence
    """
    if use_segmentation and U2NET_AVAILABLE:
        img = segment_person(image_path)
    else:
        img = Image.open(image_path).convert('RGB')
    
    # Resize and normalize
    img = img.resize((224,224))
    img_array = np.array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predict
    preds = model.predict(img_array)
    class_idx = np.argmax(preds)
    confidence = preds[0][class_idx]
    
    return class_labels[class_idx], confidence

# ----------------------------
# Test Example
# ----------------------------
if __name__ == "__main__":
    image_path = "hdi1.jpg"  # Replace with your test image
    label, conf = classify_body_shape(image_path, use_segmentation=False)  # Set True to use U2Net
    print(f"Predicted body shape: {label} (confidence: {conf:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Predicted body shape: Hourglass (confidence: 0.81)
